In [ ]:
# how to instll libs, uncomment lib to install
#!pip install azure-ai-language-conversations==1.1.0
#!pip install azure-ai-textanalytics
#!pip install azure.core
#!pip install python-dotenv
#!pip install openai
# restart your kermel 

In [ ]:
# How to check your versions, uncomment to check
#!pip list | findstr azure.ai.textanalytics
#!pip list | findstr azure.ai.language.conversations
#!pip list | findstr azure.core

### Understanding document summarization

The cell below defines variable containg some text to summarize
and a path to where 3 text files to test with are located

In [ ]:
a_single_document_text= "At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI services, I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, there’s magic—what we call XYZ-code as illustrated in Figure 1—a joint representation to create more powerful AI that can speak, hear, see, and understand humans better. We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities and languages. The goal is to have pre-trained models that can jointly learn representations to support a broad range of downstream AI tasks, much in the way humans do today. Over the past five years, we have achieved human performance on benchmarks in conversational speech recognition, machine translation, conversational question answering, machine reading comprehension, and image captioning. These five breakthroughs provided us with strong signals toward our more ambitious aspiration to produce a leap in AI capabilities, achieving multi-sensory and multilingual learning that is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational component of this aspiration, if grounded with external knowledge sources in the downstream AI tasks."
# This set is already an array
a_set_of_documents = [
        "The extractive summarization feature uses natural language processing techniques to locate key sentences in an unstructured text document. These sentences collectively convey the main idea of the document. This feature is provided as an API for developers. " ,
        "They can use it to build intelligent solutions based on the relevant information extracted to support various use cases. Extractive summarization supports several languages. It is based on pretrained multilingual transformer models, part of our quest for holistic representations. ",
        "It draws its strength from transfer learning across monolingual. It also and harness the shared nature of languages to produce models of improved quality and efficiency. "
    ]
a_conversation=["Agent: Hello, you're chatting with Rene. How may I help you?"
                "Customer: Hi, I tried to set up wifi connection for Smart Brew 300 espresso machine, but it didn't work."
                "Agent: I’m sorry to hear that. Let’s see what we can do to fix this issue. Could you please try the following steps for me? First, could you push the wifi connection button, hold for 3 seconds, then let me know if the power light is slowly blinking on and off every second?"
                "Customer: Yes, I pushed the wifi connection button, and now the power light is slowly blinking."
                "Agent: Great. Thank you! Now, please check in your Contoso Coffee app. Does it prompt to ask you to connect with the machine?"
                "Customer: No. Nothing happened."
                "Agent: I’m very sorry to hear that. Let me see if there’s another way to fix the issue. Please hold on for a minute."]
a_multi_document = [
       'We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! '
       'They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) '
       'and he is super nice, coming out of the kitchen and greeted us all.'
       ,

       'We enjoyed very much dining in the place! '
       'The Sirloin steak I ordered was tender and juicy, and the place was impeccably clean. You can even pre-order from their '
       'online menu at www.contososteakhouse.com, call 312-555-0176 or send email to order@contososteakhouse.com! '
       'The only complaint I have is the food didn\'t come fast enough. Overall I highly recommend it!'
   ]

My_text_files_path=r"C:\Repo\LearnSummarizationA\Summarization\text-analysis\txt_files"
print("Done!")

#### Import the needed libraries and environment variables

In [ ]:
import os
from azure.ai.textanalytics  import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient
from dotenv import load_dotenv 
import glob
import re
import shutil

# This example requires environment variables named "LANGUAGE_KEY" and "LANGUAGE_ENDPOINT"
load_dotenv()
key = os.environ.get('LANGUAGE_KEY')
endpoint = os.environ.get('LANGUAGE_ENDPOINT')
open_ai_key = os.environ.get('OPENAI_API_KEY')
open_ai_endpoint = os.environ.get('OPEN_AI_ENDPOINT')

# for debugging only
#print("key = ", key)
#print("endpoint = ", endpoint)

#### Create some functions to use across our testing

In [ ]:
# A function to craete and authenticate a textanalytics client
# using your key and endpoint 
# Returns the authenticated conversation_analysis_client
def authenticate_ConversationAnalysisClient_client():
    ta_credential = AzureKeyCredential(key)
    conversation_analysis_client = ConversationAnalysisClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return conversation_analysis_client

# A function to craete and authenticate a ConversationAnalysisClient client
# using your key and endpoint 
# Returns the authenticated TextAnalyticsClient
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

# A function to create an extractive summary of documennt text
# The summary is limited to x number of sentences
# Returns the summrized text
def sample_extractive_summarization(client, doocuments, max_sentences):
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import (
        TextAnalyticsClient,
        ExtractiveSummaryAction
    ) 
    
    poller = client.begin_analyze_actions(
        doocuments,
        actions=[
            ExtractiveSummaryAction(max_sentence_count=max_sentences, api_version="2022-05-15-preview"),
        ], 
    )
    summary_text=""
    all_summary_text=[]
    extract_summary_results = poller.result()
    for results in extract_summary_results:
        summary_text=""
        for result in results:
            if result.kind == "ExtractiveSummarization":
                summary_text=summary_text+ " ".join([sentence.text for sentence in result.sentences])
            elif result.is_error is True:
                print("...Is an error with code '{}' and message '{}'".format(
                    result.error.code, result.error.message
                ))
        all_summary_text==all_summary_text.append(summary_text)
    return all_summary_text


# A function to create an abstractive summary of documennt text
# The summary is limited to x number of sentences
# Returns the summrized text
def sample_abstractive_summarization(client, documents, max_sentences):
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import (
        TextAnalyticsClient,
        AbstractiveSummaryAction
    ) 
    
    poller = client.begin_analyze_actions(
        documents,
        actions=[
            AbstractiveSummaryAction(max_sentence_count=max_sentences, api_version="2022-05-15-preview"),
        ], 
    )
    summary_text=""
    all_summary_text=[]
    extract_summary_results = poller.result()
    for results in extract_summary_results:
        summary_text=""
        for result in results:
            if result.kind == "AbstractiveSummarization":
                summary_text=summary_text+ " ".join([sentence.text for sentence in result.summaries])
            elif result.is_error is True:
                print("...Is an error with code '{}' and message '{}'".format(
                    result.error.code, result.error.message
                ))
        all_summary_text==all_summary_text.append(summary_text)
    return all_summary_text


##  Extractive Summarization

### Now we will use the python SDK (specificallly the TextAnalyticsClient) to perform Extractive Summarization

This is an asychronous call

Be sure you have environment variables set for the LANGUAGE_KEY and LANGUAGE_ENDPOINT in the .env file

#### We will perform 3 scenarios:
1. Summarizing a single document to no more than 4 sentences 
2. Summarizing a set of set of document text as a batch, each to no more than 1 sentence each
3. Summarizing each individual text file in a directory

In [ ]:
# Main
#
# create and authenticate a textanalytics client
client = authenticate_client()
#
# *******************************************************************
# Scenario #1 summarize a single document to no more than 4 sentences     
# *******************************************************************           
document=a_single_document_text
#
# always pass an array of documents even if just 1 sentence in array
documents=[]
# append the 1 document to the array
documents.append(document)
#
# limit the number of sentences the summary will contain
max_sentences=4
#
# call the function to summarize the document
document_summary = sample_extractive_summarization(client, documents, max_sentences)
#
print(document_summary)
print("*************************************")
print("sumarizing a single document is done!")
print("*************************************")
#
# *******************************************************************
# summarize a set of set of documents, each to no more than 1 sentence each
# *******************************************************************
documents= a_set_of_documents
#
# summarzing to a single sentence
max_sentences=1
#
# call the function to summarize the documents
documents_summaries = sample_extractive_summarization(client, documents, max_sentences)

# loop through each summary in the returned documents_summaries
# and print each one, prepending a label: "document summary # x"
# where x is the ordinal number in the documents_summaries array
for i, summary in enumerate(documents_summaries):
    print(" **** document summary #", i+1, ": ", summary + " ****" + '\n')

#
print("***************************************")
print ("Summarizing an array of documents done!")
print("***************************************" +'\n')
print()
#
# *********************************************
# summarize a directory containing multiple txt files
# summarize each file in the directory and
# write the summary to a new file with the same
# name as the original file but with a -summary
# at the end of the file name (another .txt file)
# Put the summary files in a difernt directory
# **********************************************
#
# get a list of all the files in the directory that have a .txt extension
mypath=My_text_files_path + "\*.txt"
print(mypath)
#
# create a list called onlytxtfiles of just the txt files in the direactory
onlytxtfiles=glob.glob(mypath)
#
# print a count of how many txt files are in the collection
print("There are ", len(onlytxtfiles), " files in the directory")
#
# Then start processing them in a loop
print("processing files..." +'\n')
#
#loop through the txt files
i=0
# set the number of sentences to sumarize the text into
max_sentences=4
#
for file in onlytxtfiles:
    i=i+1
    with open(file, 'r') as myfile:
        data=myfile.read().replace('\n', '')
        # always pass an array of documents even if just 1 documents in array
        documents=[]
        documents.append(data)
        # call the function to summarize the document
        document_summary = sample_extractive_summarization(client, documents, max_sentences)
        # write the summary to a new file with the same name as the original file
        # in a different directory
        # but with a -summary at the end of the file name
        newfile=file.replace(".txt", "-summary.txt")
        newfile=newfile.replace("txt_files","ExtractiveSummaries")
        # we are overwriting if they exist!
        with open(newfile, 'w') as f:
            f.write(str(document_summary[0]))
            f.close()
            print ("file written: ", newfile)
            print(" **** file # ", i, " is done ****" '\n')
#
print("***************************************************")
print("Done with processing individual files in a directory" +'\n')
print("***************************************************" +'\n')

print("******************************" + '\n')
print("Done with all 3 scenarios!" + '\n')
print("******************************")


##  Abstractive Summarization

### Now we will use the python SDK (specificallly the TextAnalyticsClient) to perform Abstractive Summarization

This is an asychronous call

Be sure you have environment variables set for the LANGUAGE_KEY and LANGUAGE_ENDPOINT in the .env file

#### We will perform just 1 scenario
1. Summarizing each individual text file in a directory

In [ ]:
# *********************************************
# summarize a directory containing multiple txt files
# summarize each file in the directory and
# write the summary to a new file with the same
# name as the original file but with a -summary
# at the end of the file name (another .txt file)
# Put the summary files in a difernt directory
# **********************************************
#
# create and authenticate a textanalytics client
client = authenticate_client()
#
# get a list of all the files in the directory that have a .txt extension
mypath=My_text_files_path + "\*.txt"
print(mypath)
#
# create a list called onlytxtfiles of just the txt files in the direactory
onlytxtfiles=glob.glob(mypath)
#
# print a count of how many txt files are in the collection
print("There are ", len(onlytxtfiles), " files in the directory")
#
# Then start processing them in a loop
print("processing files..." +'\n')
#
#loop through the txt files
i=0
# set the number of sentences to sumarize the text into
max_sentences=4
#
for file in onlytxtfiles:
    i=i+1
    with open(file, 'r') as myfile:
        data=myfile.read().replace('\n', '')
        # always pass an array of documents even if just 1 documents in array
        documents=[]
        documents.append(data)
        # call the function to summarize the document
        document_summary = sample_abstractive_summarization(client, documents, max_sentences)
        # write the summary to a new file with the same name as the original file
        # in a different directory
        # but with a -summary at the end of the file name
        newfile=file.replace(".txt", "-summary.txt")
        newfile=newfile.replace("txt_files","AbstractiveSummaries")
        # we are overwriting if they exist!
        with open(newfile, 'w') as f:
            f.write(str(document_summary[0]))
            f.close()
            print ("file written: ", newfile)
            print(" **** file # ", i, " is done ****" '\n')
print("******************************************************************************************")
print("Done with generating abstractive summaries for the  individual files in a directory" +'\n')
print("************************************************************************************" +'\n')

print("**************************************************" + '\n')
print("Done with the 1 abstractive summary scenario!" + '\n')
print("*************************************************")

## Using your data - Extractive Summarization

### Now use extractive summarization to summarize your call center text files

The code assumes they have a .txt extension. Change if necessary.  
Also change the max_sentences value as desired.

#### We will perform just 1 scenario
1. Create an Extractive Summarization each your individual text file in a directory

Change the path in the cell below to be where your text files are located.  
Also provide a path to where you want the summaries to be written


In [ ]:
# ***************************************************************************************
# Change the My_text_files_path variable in the line below to the path to your text files
# ***************************************************************************************
My_text_files_path=r"C:\the local path to your files (with no slash at the end)"
#My_text_files_path=r"C:\Repo\LearnSummarization\Summarization\text-analysis\txt_files"
#
# the following cells will make use of the specified path

In [ ]:
# create and authenticate a textanalytics client
client = authenticate_client()
#
# get a list of all the files in the directory that have a .txt extension
# change if needed
mypath=My_text_files_path + "\*.txt"
#print(mypath)
#
# create a list called onlytxtfiles of just the txt files in the direactory
# looking for .txt files, change if needed
onlytxtfiles=glob.glob(mypath)

# print a count of how many files are in the collection
print("There are ", len(onlytxtfiles), " files in the directory")
#
print("processing files..." +'\n')
#
#loop through the .txt files, change if needed
i=0
# set the number of sentences to sumarize the text into
max_sentences=4
#
for file in onlytxtfiles:
    i=i+1
    with open(file, 'r') as myfile:
        data=myfile.read().replace('\n', '')
        # always pass an array of documents even if just 1 documents in array
        documents=[]
        documents.append(data)
        # call the function to summarize the document
        document_summary = sample_extractive_summarization(client, documents, max_sentences)
        # write the summary to a new file with the same name as the original file
        # in a different directory
        # but with a -summary at the end of the file name
        newfile=file.replace(".txt", "-summary.txt")
        newfile=newfile.replace("txt_files","ExtractiveSummaries")
        # we are overwriting if they exist!
        with open(newfile, 'w') as f:
            f.write(str(document_summary[0]))
            f.close()
            print ("file written: ", newfile)
            print(" **** file # ", i, " is done ****" '\n')
print("*********************************************************")
print("Done with processing your individual files in a directory" +'\n')
print("*********************************************************" +'\n')
#
print("*************************************************************" + '\n')
print("Done with the Extractive Summarization scenario on your data!" + '\n')
print("*************************************************************")


**Now for an *abstractive* ....**

## Using your data - Abstractive Summarization

### Now use abstractive summarization to summarize your call center text files

The code assumes they have a .txt extension. Change if necessary.  
Also change the max_sentences value as desired.

#### We will perform just 1 scenario
1. Create an Abstractive Summarization each your individual text file in a directory



In [ ]:
# create and authenticate a textanalytics client
client = authenticate_client()
#
# *****************************************************************
# Change the pathe in the line below to the path to your text files (no slash at end)
# *****************************************************************
#My_text_files_path=r"C:\the local path to your files"
My_text_files_path=r"C:\Repo\LearnSummarization\Summarization\text-analysis\txt_files"
#
# get a list of all the files in the directory that have a .txt extension
# change if needed

mypath=My_text_files_path + "\*.txt"
print(mypath)
#
# create a list called onlytxtfiles of just the txt files in the direactory
# looking for .txt files, change if needed
onlytxtfiles=glob.glob(mypath)

# print a count of how many files are in the collection
print("There are ", len(onlytxtfiles), " files in the directory")
print("processing files..." +'\n')
#
#loop through the .txt files, change if needed
i=0
# set the number sentences to sumarize the text into
max_sentences=4
#
for file in onlytxtfiles:
    i=i+1
    with open(file, 'r') as myfile:
        data=myfile.read().replace('\n', '')
        # always pass an array of documents even if just 1 documents in array
        documents=[]
        documents.append(data)
        # call the function to summarize the document
        document_summary = sample_abstractive_summarization(client, documents, max_sentences)
        # write the summary to a new file with the same name as the original file
        # in a different directory
        # but with a -summary at the end of the file name
        newfile=file.replace(".txt", "-summary.txt")
        newfile=newfile.replace("txt_files","AbstractiveSummaries")
        # we are overwriting if they exist!
        with open(newfile, 'w') as f:
            f.write(str(document_summary[0]))
            f.close()
            print ("file written: ", newfile)
            print(" **** file # ", i, " is done ****" '\n')
print("*********************************************************")
print("Done with processing your individual files in a directory" +'\n')
print("*********************************************************" +'\n')
#
print("*************************************************************" + '\n')
print("Done with the Extractive Summarization scenario on your data!" + '\n')
print("*************************************************************")

### Review your summary files!

### Which summarization type is what you want, **Extractive or Abstractive**?

### Which did a better summarization,  the **Azure Open Service** *or* the **Language Service**?

# =======================================================================================================================

## Azure Open AI
### Could it do as good of a job at summarization

Let's try! I'll the C:\Repo\LearnSummarization\Summarization\text-analysis\txt_files\sample1.txt file

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
openai.api_type = "azure"
# Change the endpoint
openai.api_base = "https://<your endpoint>.openai.azure.com/"
#
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.ChatCompletion.create(
  engine="<your depployment name>",
  messages = [{"role":"system","content":"You are an AI assistant whos job is to summarize text.You will be giveven some text and your should perform an abstractive summarization. Summarize the text into no more than 4 sentences. Avoid making the sentences too long."},
              {"role":"user","content":"Text to summarize: At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI services, I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, there’s magic—what we call XYZ-code as illustrated in Figure 1—a joint representation to create more powerful AI that can speak, hear, see, and understand humans better. We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities and languages. The goal is to have pre-trained models that can jointly learn representations to support a broad range of downstream AI tasks, much in the way humans do today. Over the past five years, we have achieved human performance on benchmarks in conversational speech recognition, machine translation, conversational question answering, machine reading comprehension, and image captioning. These five breakthroughs provided us with strong signals toward our more ambitious aspiration to produce a leap in AI capabilities, achieving multi-sensory and multilingual learning that is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational component of this aspiration, if grounded with external knowledge sources in the downstream AI tasks."}],
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)
print(response["choices"][0]['message']["content"]) 



# =======================================================================================================================

## Next Topic

### But what if you wanted other features of the Language Service like:
* Detecting the language
* Identifying the key phrases
* Identifying Sentiment
* Extracting entities
* Extracting linked entities



## Sentiment Analysis

In [ ]:
def sample_analyze_sentiment() -> None:
    print(
        "In this sample we will be combing through reviews customers have left about their"
        "experience using our skydiving company, Contoso."
    )
    print("****************************************************************************************")
    print(
        "We start out with a list of reviews. Let us extract the reviews we are sure are "
        "positive, so we can display them on our website and get even more customers!"
    )
    print("****************************************************************************************")
    # [START analyze_sentiment]
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    # create and authenticate a textanalytics client
    text_analytics_client = authenticate_client()

    documents = [
        """I had the best day of my life. I decided to go sky-diving and it made me appreciate my whole life so much more.
        I developed a deep-connection with my instructor as well, and I feel as if I've made a life-long friend in her.""",
        """This was a waste of my time. All of the views on this drop are extremely boring, all I saw was grass. 0/10 would
        not recommend to any divers, even first timers.""",
        """This was pretty good! The sights were ok, and I had fun with my instructors! Can't complain too much about my experience""",
        """I only have one word for my experience: WOW!!! I can't believe I have had such a wonderful skydiving company right
        in my backyard this whole time! I will definitely be a repeat customer, and I want to take my grandmother skydiving too,
        I know she'll love it!"""
    ]


    result = text_analytics_client.analyze_sentiment(documents, show_opinion_mining=True)
    docs = [doc for doc in result if not doc.is_error]
    print("Let's visualize the sentiment of each of these documents")
    print("****************************************************************************************" + '\n')
    for idx, doc in enumerate(docs):
        print('\t' + f"Document text: {documents[idx]}"+ '\n')
        print('\t' + f"Overall sentiment: {doc.sentiment}" + '\n')
        print("****************************************************************************************" )
    print("Now, let us extract all of the positive reviews")
    positive_reviews = [doc for doc in docs if doc.sentiment == 'positive']
    print("We want to be very confident that our reviews are positive since we'll be posting them on our website.")
    print("We're going to confirm our chosen reviews are positive using two different tests")
    print("****************************************************************************************")
    print(
        "**** Test #1 ****"
        "First, we are going to check how confident the sentiment analysis model is that a document is positive. "
        "Let's go with a 90% confidence."
    )
    # filter fr only those with a high confidence score
    positive_reviews = [
        review for review in positive_reviews
        if review.confidence_scores.positive >= 0.9
    ]
    print("Done! There are now {} positive reviews with a confidence score of at least 90%".format(len(positive_reviews)))
    print( 
        "**** Test #2 ****"
        "Finally, we also want to make sure every sentence is positive so we only showcase our best selves!" + '\n'
    )
    # we filter the filtered set sentence by sentence
    positive_reviews_final = []
    for idx, review in enumerate(positive_reviews):
        print(f"Looking at positive review #{idx + 1}")
        any_sentence_not_positive = False
        for sentence in review.sentences:
            print("...Sentence '{}' has sentiment '{}' with confidence scores '{}'".format(
                sentence.text,
                sentence.sentiment,
                sentence.confidence_scores
                )
            )
            if sentence.sentiment != 'positive':
                any_sentence_not_positive = True
        if not any_sentence_not_positive:
            positive_reviews_final.append(review)
    print("****************************************************************************************" +'\n')
    print("We now have the final list of positive reviews we are going to display on our website!" + '\n')


# Main
sample_analyze_sentiment()

## Detect Language

This sample demonstrates how to detect language in a batch of different documents.

In this sample, we own a hotel with a lot of international clientele. We are looking to catalog the reviews we have for our hotel by language, so  we can translate these reviews into English.

In [ ]:
def sample_detect_language() -> None:
    print("*************************************************************************************************************" + '\n')
    print(
        "In this sample we own a hotel with customers from all around the globe. We want to eventually\n"
        "translate these reviews into English so our manager can read them. However, we first need to know which language\n"
        "they are in for more accurate translation. This is the step we will be covering in this sample\n"
    )
    print("*************************************************************************************************************" + '\n')
    # [START detect_language]
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    # create and authenticate a textanalytics client
    text_analytics_client = authenticate_client()

    documents = [
        """
        The concierge Paulette was extremely helpful. Sadly when we arrived the elevator was broken, but with Paulette's help we barely noticed this inconvenience.
        She arranged for our baggage to be brought up to our room with no extra charge and gave us a free meal to refurbish all of the calories we lost from
        walking up the stairs :). Can't say enough good things about my experience!
        """,
        """
        最近由于工作压力太大，我们决定去富酒店度假。那儿的温泉实在太舒服了，我跟我丈夫都完全恢复了工作前的青春精神！加油！
        """
    ]

    result = text_analytics_client.detect_language(documents)
    reviewed_docs = [doc for doc in result if not doc.is_error]

    print("Let's see what language each review is in!")
    print("*************************************************************************************************************" + '\n')

    for idx, doc in enumerate(reviewed_docs):
        print("\tReview #{} is in '{}', which has ISO639-1 name '{}'\n".format(
            idx, doc.primary_language.name, doc.primary_language.iso6391_name
        ))
        print("*************************************************************************************************************" + '\n')
    # [END detect_language]
    print(
        "When actually storing the reviews, we want to map the review to their ISO639-1 name "
        "so everything is more standardized"
    )

    review_to_language = {}
    for idx, doc in enumerate(reviewed_docs):
        review_to_language[documents[idx]] = doc.primary_language.iso6391_name


sample_detect_language()

## Key Phrases
This sample demonstrates how to extract key talking points from a batch of documents.

In this sample, we want to go over articles and read the ones that mention Microsoft.
We're going to use the SDK to create a rudimentary search algorithm to find these articles.

In [ ]:
def sample_extract_key_phrases() -> None:
    print("*************************************************************************************************************" + '\n')
    print(
        "In this sample, we want to find the articles that mention Microsoft to read.\n"
    )
    print("*************************************************************************************************************" + '\n')
    articles_that_mention_microsoft = []
    #
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    # create and authenticate a textanalytics client
    text_analytics_client = authenticate_client()
    
    articles = [
        """
        Washington, D.C. Autumn in DC is a uniquely beautiful season. The leaves fall from the trees
        in a city chock-full of forests, leaving yellow leaves on the ground and a clearer view of the
        blue sky above...
        """,
        """
        Redmond, WA. In the past few days, Microsoft has decided to further postpone the start date of
        its United States workers, due to the pandemic that rages with no end in sight...
        """,
        """
        Redmond, WA. Employees at Microsoft can be excited about the new coffee shop that will open on campus
        once workers no longer have to work remotely...
        """
    ]

    result = text_analytics_client.extract_key_phrases(articles)
    for idx, doc in enumerate(result):
        if not doc.is_error:
            print("Key phrases in article #{}: {}".format(
                idx + 1,
                ", ".join(doc.key_phrases)
            ))
    # Shows how to filter out the articles that mention Microsoft
            if "Microsoft" in doc.key_phrases:
                articles_that_mention_microsoft.append(str(idx + 1))
    print("\n*************************************************************************************************************" + '\n')
    print(
        "The articles that mention Microsoft are articles number: {}. Those are the ones I'm interested in reading.".format(
            ", ".join(articles_that_mention_microsoft)
        )
    )


sample_extract_key_phrases()

## Recognize PII

This sample demonstrates how to recognize personally identifiable information in a batch of documents.
The endpoint recognize_pii_entities is only available for API version v3.1 and up.

In this sample, we will be working for a company that handles loan payments. To follow privacy guidelines,
we need to redact all of our information before we make it public.

In [ ]:
def sample_recognize_pii_entities() -> None:
    print("\n*************************************************************************************************************" + '\n')
    print(
        "In this sample we will be going through our customer's loan payment information and redacting\n"
        "all PII (personally identifiable information) before storing this information on our public website.\n"
        "I'm also looking to explicitly extract the SSN information, so I can update my database with SSNs for\n"
        "our customers"
    )
    print("\n*************************************************************************************************************" + '\n')
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    # create and authenticate a textanalytics client
    text_analytics_client = authenticate_client()
    
    documents = [
        """Parker Doe has repaid all of their loans as of 2020-04-25.
        Their SSN is 859-98-0987. To contact them, use their phone number
        555-555-5555. They are originally from Brazil and have Brazilian CPF number 998.214.865-68"""
    ]

    result = text_analytics_client.recognize_pii_entities(documents)
    docs = [doc for doc in result if not doc.is_error]

    print(
        "Let's compare the original document with the documents after redaction. "
        "I also want to comb through all of the entities that got redacted"
    )
    for idx, doc in enumerate(docs):
        print("\n*************************************************************************************************************" + '\n')
        print(f"Document text: {documents[idx]}\n")
        print(f"Redacted document text: {doc.redacted_text}\n")
        print("\t*************************************************************************************************************" + '\n')
        for entity in doc.entities:
            print("\t...Entity '{}' with category '{}' got redacted".format(
                entity.text, entity.category
            ))
            print("\n\t*************************************************************************************************************" + '\n')

    print("All of the information that I expect to be redacted is!")

    print(
        "Now I want to explicitly extract SSN information to add to my user SSN database. "
        "I also want to be fairly confident that what I'm storing is an SSN, so let's also "
        "ensure that we're > 60% positive the entity is a SSN"
    )
    social_security_numbers = []
    for doc in docs:
        for entity in doc.entities:
            # showing a filter
            if entity.category == 'USSocialSecurityNumber' and entity.confidence_score >= 0.6:
                social_security_numbers.append(entity.text)
    print("\n\t*************************************************************************************************************" + '\n')
    print("\tWe have extracted the following SSNs: '{}'".format(
        "', '".join(social_security_numbers)
    ))
    print("\n\t*************************************************************************************************************" + '\n')


# Main
sample_recognize_pii_entities()

## Healthcare Entities
This sample demonstrates how to detect healthcare entities in a batch of documents.  

In this sample we will be a newly-hired engineer working in a pharmacy. We are going to  
comb through all of the prescriptions our pharmacy has fulfilled so we can catalog how  
much inventory we have.

In [ ]:
def sample_analyze_healthcare_entities() -> None:
    import os
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient, HealthcareEntityRelation
    print(
        "In this sample we will be combing through the prescriptions our pharmacy has fulfilled "
        "so we can catalog how much inventory we have"
    )
    print(
        "We start out with a list of prescription documents.\n"
    )

    # create and authenticate a textanalytics client
    text_analytics_client = authenticate_client()

    documents = [
        """Patient needs to take 100 mg of ibuprofen, and 3 mg of potassium. Also needs to take 10 mg of Zocor.""",
        """Patient needs to take 50 mg of ibuprofen, and 2 mg of Coumadin."""
    ]
    print(documents,)
    poller = text_analytics_client.begin_analyze_healthcare_entities(documents)
    result = poller.result()

    docs = [doc for doc in result if not doc.is_error]
    print("*************************************************************************************************************" + '\n')
    print("Let's first visualize the outputted healthcare result:\n")
    print("*************************************************************************************************************" + '\n')
    for doc in docs:
        for entity in doc.entities:
            print(f"Entity: {entity.text}")
            print(f"...Normalized Text: {entity.normalized_text}")
            print(f"...Category: {entity.category}")
            print(f"...Subcategory: {entity.subcategory}")
            print(f"...Offset: {entity.offset}")
            print(f"...Confidence score: {entity.confidence_score}")
            if entity.data_sources is not None:
                print("...Data Sources:")
                for data_source in entity.data_sources:
                    print(f"......Entity ID: {data_source.entity_id}")
                    print(f"......Name: {data_source.name}")
            if entity.assertion is not None:
                print("...Assertion:")
                print(f"......Conditionality: {entity.assertion.conditionality}")
                print(f"......Certainty: {entity.assertion.certainty}")
                print(f"......Association: {entity.assertion.association}")
        for relation in doc.entity_relations:
            print(f"Relation of type: {relation.relation_type} has the following roles")
            for role in relation.roles:
                print(f"...Role '{role.name}' with entity '{role.entity.text}'")
        print("------------------------------------------")

    print("Now, let's get all of medication dosage relations from the documents")
    dosage_of_medication_relations = [
        entity_relation
        for doc in docs
        for entity_relation in doc.entity_relations if entity_relation.relation_type == HealthcareEntityRelation.DOSAGE_OF_MEDICATION
    ]
    # [END analyze_healthcare_entities]

    print(
        "\tDone!\n"
        "Now, I will create a dictionary of medication to total dosage. "
        "I will use a regex to extract the dosage amount. For simplicity sake, I will assume "
        "all dosages are represented with numbers and have mg unit."
    )
    import re
    from collections import defaultdict

    medication_to_dosage: dict[str, int] = defaultdict(int)

    for relation in dosage_of_medication_relations:
        # The DosageOfMedication relation should only contain the dosage and medication roles

        dosage_role = next(iter(filter(lambda x: x.name == "Dosage", relation.roles)))
        medication_role = next(iter(filter(lambda x: x.name == "Medication", relation.roles)))

        try:
            dosage_value = int(re.findall(r"\d+", dosage_role.entity.text)[0]) # we find the numbers in the dosage
            medication_to_dosage[medication_role.entity.text] += dosage_value
        except StopIteration:
            # Error handling for if there's no dosage in numbers.
            pass

    for medication, dosage in medication_to_dosage.items():
        print("\tWe have fulfilled '{}' total mg of '{}'".format(
            dosage, medication
        ))


# Main
sample_analyze_healthcare_entities()

## Custom Entities

This sample demonstrates how to recognize custom entities in documents.
Recognizing custom entities is also available as an action type through the begin_analyze_actions API.

For information on regional support of custom features and how to train a model to
recognize custom entities, see https://aka.ms/azsdk/textanalytics/customentityrecognition

You need to have trained a custom model before this sample will work.  
you can do it using Language Studio  
see: https://learn.microsoft.com/en-us/azure/ai-services/language-service/custom-named-entity-recognition/quickstart?pivots=language-studio


In [ ]:
def sample_recognize_custom_entities() -> None:
    import os
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    #project_name = os.environ["CUSTOM_ENTITIES_PROJECT_NAME"]
    #deployment_name = os.environ["CUSTOM_ENTITIES_DEPLOYMENT_NAME"]
    #
    # The below values come from Language Studio
    project_name = "<your project name>"
    deployment_name = "<Your model deployment name>"
    #
    path_to_sample_document =  r"C:\Repo\LearnSummarization\Summarization\text-analysis\CustomEntities\custom_entities_sample.txt"
    print(path_to_sample_document)
    #
    # create and authenticate a textanalytics client
    text_analytics_client = authenticate_client()

    with open(path_to_sample_document) as fd:
        document = [fd.read()]

    poller = text_analytics_client.begin_recognize_custom_entities(
        document,
        project_name=project_name,
        deployment_name=deployment_name
    )

    document_results = poller.result()
    for custom_entities_result in document_results:
        if custom_entities_result.kind == "CustomEntityRecognition":
            for entity in custom_entities_result.entities:
                print(
                    "Entity '{}' has category '{}' with confidence score of '{}'".format(
                        entity.text, entity.category, entity.confidence_score
                    )
                )
        elif custom_entities_result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                custom_entities_result.code, custom_entities_result.message
                )
            )

# Main# How to check your versions
sample_recognize_custom_entities()

### Performing multiple actions on a single document 

#### We will perform just 1 scenario but it will do several things
1. Perform the following tasks:
    Recognize Entities. Recognize LinkedEntities, Recognize PiiEntities, Extract KeyPhrases and Analyze Sentiment


In [ ]:
def sample_recognize_custom_entities() -> None:
   import os
   from azure.core.credentials import AzureKeyCredential
   from azure.ai.textanalytics import (
    TextAnalyticsClient,
    RecognizeEntitiesAction,
    RecognizeLinkedEntitiesAction,
    RecognizePiiEntitiesAction,
    ExtractKeyPhrasesAction,
    AnalyzeSentimentAction,
    )
   endpoint = os.environ["LANGUAGE_ENDPOINT"]
   key = os.environ["LANGUAGE_KEY"]
   print(endpoint)
   print(key)
   text_analytics_client = TextAnalyticsClient(
       endpoint=endpoint,
       credential=AzureKeyCredential(key),
       )

   documents = [
       'We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! '
       'They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) '
       'and he is super nice, coming out of the kitchen and greeted us all.'
       ,
       
       'We enjoyed very much dining in the place! '
       'The Sirloin steak I ordered was tender and juicy, and the place was impeccably clean. You can even pre-order from their '
       'online menu at www.contososteakhouse.com, call 312-555-0176 or send email to order@contososteakhouse.com! '
       'The only complaint I have is the food didn\'t come fast enough. Overall I highly recommend it!'
       ]
   poller = text_analytics_client.begin_analyze_actions(
       documents,
       display_name="Sample Text Analysis",
    actions=[
        RecognizeEntitiesAction(),
        AnalyzeSentimentAction(),
        ],
    )
   document_results = poller.result()
   for doc, action_results in zip(documents, document_results):
        print(f"\nDocument text: {doc}")
        for result in action_results:
            if result.kind == "EntityRecognition":
                print("...Results of Recognize Entities Action:")
                for entity in result.entities:
                    print(f"......Entity: {entity.text}")
                    print(f".........Category: {entity.category}")
                    print(f".........Confidence Score: {entity.confidence_score}")
                    print(f".........Offset: {entity.offset}")
            elif result.kind == "PiiEntityRecognition":
                print("...Results of Recognize PII Entities action:")
                for pii_entity in result.entities:
                    print(f"......Entity: {pii_entity.text}")
                    print(f".........Category: {pii_entity.category}")
                    print(f".........Confidence Score: {pii_entity.confidence_score}")
            elif result.kind == "KeyPhraseExtraction":
                print("...Results of Extract Key Phrases action:")
                print(f"......Key Phrases: {result.key_phrases}")
            elif result.kind == "EntityLinking":
                print("...Results of Recognize Linked Entities action:")
                for linked_entity in result.entities:
                    print(f"......Entity name: {linked_entity.name}")
                    print(f".........Data source: {linked_entity.data_source}")
                    print(f".........Data source language: {linked_entity.language}")
                    print(
                        f".........Data source entity ID: {linked_entity.data_source_entity_id}"
                        )
                    print(f".........Data source URL: {linked_entity.url}")
                    print(".........Document matches:")
                    for match in linked_entity.matches:
                        print(f"............Match text: {match.text}")
                        print(f"............Confidence Score: {match.confidence_score}")
                        print(f"............Offset: {match.offset}")
                        print(f"............Length: {match.length}")
            elif result.kind == "SentimentAnalysis":
                print("...Results of Analyze Sentiment action:")
                print(f"......Overall sentiment: {result.sentiment}")
                print(
                    f"......Scores: positive={result.confidence_scores.positive}; \
                        neutral={result.confidence_scores.neutral}; \
                            negative={result.confidence_scores.negative} \n"
                            )
            elif result.is_error is True:
                print(
                    f"...Is an error with code '{result.error.code}' and message '{result.error.message}'"
                    )
                print("------------------------------------------")


# Main
sample_recognize_custom_entities()
